In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import NaN

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **1. Read file**

In [ ]:
book_df = pd.read_csv('../input/book-recommendation-dataset/Books.csv')
ratings_df = pd.read_csv('../input/book-recommendation-dataset/Ratings.csv').sample(40000)
user_df = pd.read_csv('../input/book-recommendation-dataset/Users.csv')

In [ ]:
book_df.head(2)

In [ ]:
ratings_df.sample(2)

In [ ]:
user_df.head(2)

In [ ]:
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

In [ ]:
user_rating_df.head(2).fillna(0)

In [ ]:
book_mat = user_rating_df.pivot_table(index = 'User-ID', columns = 'ISBN', values = 'Book-Rating').replace({NaN:-1})

In [ ]:
book_mat

# Calculate rating for each Book

In [ ]:
ratings = pd.DataFrame(user_rating_df.groupby('ISBN')['Book-Rating'].mean())
ratings['num_of_ratings'] = pd.DataFrame(user_rating_df.groupby('ISBN')['Book-Rating'].count())
#ratings.sort_values('num_of_ratings', ascending = False)
ratings.merge(book_df[['ISBN','Book-Title']], left_on = 'ISBN', right_on = 'ISBN').sort_values('num_of_ratings', ascending = False)

# Select target book

In [ ]:
book_target_user_ratings = book_mat['0316666343']

# Calculate correlation value between target book to other

In [ ]:
similar_to_book_target = book_mat.corrwith(book_target_user_ratings)
corr_book_target = pd.DataFrame(similar_to_book_target, columns = ['Correlation'])
#corr_book_target.dropna(inplace = True)
corr_book_target = corr_book_target.join(ratings['num_of_ratings']).join(ratings['Book-Rating'])
corr_book_target = corr_book_target.merge(book_df[['ISBN','Book-Title']], left_on = 'ISBN', right_on = 'ISBN')

# Show Book Recomendation

In [ ]:
corr_book_target[corr_book_target['num_of_ratings'] > 10].sort_values('Correlation', ascending = False).head(10)